# 第三节 检索器与召回

之前的内容里，我们讲了 RAG 的三大基本组件：**检索器**（Retriever）、**外部文档**（Document）和**大语言模型**（LLM）。RAG 这玩意儿，说白了就是让**大模型不再瞎编，通过检索外部知识来生成靠谱的答案**。

但问题是，**怎么知道你搭的 RAG 系统到底行不行？** 这就像考试，得有个评分标准。

我觉得可以从这么几个角度来看：

首先是**检索**（Retriever）这块儿。你想啊，如果检索回来的文档压根就不相关，那后面生成啥都白搭。衡量检索文档的有效性有一个指标——**召回率。** 如果召回率低 **，** 或者说无限趋近于 0 **，** 系统基本就退化成了纯大模型回答，那效果...简直没眼看。所以召回率这个指标特别关键，它直接决定了**你能给生成模型提供多少有用的"弹药"** 。

但光召回还不够。有时候你**检索回来一大堆文档，里面可能就几句话是真正有用的，其余都是噪音**。这就像你找资料写论文，翻了十本书，真正能用的可能就那么几段。

所以还得看**上下文相关性**，也就是 RAGAS（RAG Assessment，RAG 评估的缩写)里说的**Context Relevance**。这个指标能告诉你，**召回的内容里到底有多少是真金白银**。

然后是**生成**（LLM）这一块。这里有个特别有意思的问题：模型会不会"跑偏"？就是明明给了它相关的上下文，它却自己发挥想象力去了。

RAGAS 的**忠诚度**（Faithfulness）指标就是专门盯着这个的。之前用某个开源模型，忠诚度特别低，经常是给它 A，它能给你扯到 Z 去，真是让人哭笑不得。

最后当然是看**整体效果**了——生成的答案到底对不对用户的胃口。**答案相关性**（Answer Relevance）这个指标就是最终的判官。

其实做下来感觉，一个好的 RAG 系统就像一个优秀的研究助理：**检索要准**（高召回率），**筛选要精**（高上下文相关性），**理解要透**（高忠诚度），**表达要切题**（高答案相关性）。这四个维度缺一不可，任何一个短板都会拖累整体效果。

不过说实话，实际操作中要把这几个指标都调优还真不容易，经常是按下葫芦浮起瓢，提高了召回率可能引入更多噪音，优化了忠诚度可能又损失了灵活性。这种平衡的艺术，大概就是做 RAG 系统最有挑战性的地方吧。

## RAG 的重点在检索器的优化

从 RAG 的实践中可以得出这样一个结论：**RAG 的灵魂在检索，不在生成**。

怎么说呢？就像你去图书馆找资料，如果你连相关的参考文献都找不到，读书能力再强也写不出好论文啊。

我们之前就吃过这个亏，用了个特别牛的大模型，结果效果还是很差。后来仔细一查，发现**检索这块儿根本就是个筛子，该捞的没捞上来**。

提升召回率这事儿，我总结下来其实就是三板斧：

第一板斧是**查询重写**。用户的问题往往**表达得很随意**，比如**问"最新的报销政策"** ，但知识库里可能写的是"**2025 年差旅费用管理办法**"。这时候你不改写查询，怎么可能检索到？

我们试过让**大模型先理解用户意图**，然后**生成多个不同角度的查询**，效果立竿见影。有次一个同事问"**出差住宿标准**"，系统自动扩展成了"**差旅住宿费用标准**"、"**员工出差酒店报销规定**"等好几个查询，一下子把相关文档都捞出来了。

第二板斧是**检索策略本身**。这里面门道就多了。

比如说**文档怎么切分**？切太大，噪音多；切太小，上下文不全。我们试过各种**切分尺寸**（chunk size），最后发现还是得根据文档类型来。政策文件可以切大块，FAQ 就得切小点。

还有就是要不要**用向量检索配合关键词检索**？纯向量检索有时候会漏掉一些专有名词，但关键词检索又太死板。最后我们搞了个**混合检索**，两边的结果都要，然后融合排序。

第三板斧就是**重排序**了。初步检索可能捞上来几十个文档块，但真正相关的可能就那么几个。这时候需要一个更精准的模型来把真正有用的挑出来放在前面。我们用了个专门的**重排序模型**（reranker），这东西虽然慢点，但是准啊！

给你看个真实的例子。我们有个同事要查"**产假政策**"，最开始系统召回率只有 40%左右，很多相关规定都没找到，比如"**生育假期管理办法**"、"**女职工权益保护规定**"这些。

后来我们搞了**多路召回**，一路**用向量检索找语义相关的**，一路**用关键词检索找包含"产假"、"生育"、"休假"这些词的**，再加一路**用同义词扩展**。三路并行，召回率直接干到了 95%！

最神奇的是，召回率上去之后，整个系统都活了。重排序因为有了更多高质量的候选文档，准确率从 70%提升到 92%。最终的问答准确率更是从 60%飙到 88%。这就是为啥我说**RAG 的重点在 R 不在 G**——检索不行，给再好的模型也是巧妇难为无米之炊。

当然，这里面也有些坑。比如召回太多也不行，会引入噪音；重排序模型如果太重，延迟会很高。所以还是得根据实际场景找平衡。但总的来说，在检索上多花点功夫，绝对值得。

那么，召回率和上下文相关性如何计算呢？

# 召回率和上下文相关性的计算

**上下文召回率**，说白了就是看你**该找到的东西有没有都找到**。打个比方，如果正确答案**需要 5 个关键信息点**，你的**检索系统找回来了 3 个**，那召回率就是 60%。数学公式虽然看着唬人，其实道理很简单——就是算个交集除以总数嘛。当然了，召回率越高，说明**检索组件召回的信息越全面**。

**上下文相关性**，这个是看你**找回来的东西里，有多少是真正有用的**。我之前碰到过一个特别搞笑的例子，问"**苹果手机的电池容量**"，结果系统把关于"**苹果营养价值**"的内容也召回来了。虽然都有"苹果"这个词，但完全是两码事儿。这种情况下，相关性就很低了。同样的，上下文相关性越高，说明**检索组件检索到的上下文与查询的语义相关性越强**。

实际操作的时候，LazyLLM 这个工具还挺好用的。你把问题、标准答案、检索到的内容，还有应该检索到的内容都准备好，几行代码就能算出这些指标。不过有个坑要注意——如果用 LLM 来评估的话，token 消耗会比较大，钱包要做好准备。

我们通过一个简单案例模拟一下这两个指标的计算，直接上代码：

In [ ]:
pip install lazyllm
export QWEN_API_KEY='sk-这里填写你的key'

In [ ]:
import lazyllm
import os
from lazyllm.tools.eval import LLMContextRecall, NonLLMContextRecall, ContextRelevance

# 检索组件要求准备满足如下格式要求的数据进行评估
data = [
    {
        "question": "非洲的猴面包树果实的长度约是多少厘米？",
        # 当使用基于LLM的评估方法时要求答案是标注的正确答案
        "answer": "非洲猴面包树的果实长约15至20厘米。",
        # context_retrieved 为召回器召回的文档，按段落输入为列表
        "context_retrieved": [
            "非洲猴面包树是一种锦葵科猴面包树属的大型落叶乔木，原产于热带非洲，它的果实长约15至20厘米。",
            "钙含量比菠菜高50％以上，含较高的抗氧化成分。",
        ],
        # context_reference 为标注的应当被召回的段落
        "context_reference": [
            "非洲猴面包树是一种锦葵科猴面包树属的大型落叶乔木，原产于热带非洲，它的果实长约15至20厘米。"
        ],
    }
]
# 返回召回文档的命中率，例如上述data成功召回了标注的段落，因此召回率为1
m_recall = NonLLMContextRecall()
res = m_recall(data)  # 1.0
print(res)

# 返回召回文档中的上下文相关性分数，例如上述data召回的两个句子中只有一个是相关的
m_cr = ContextRelevance()
res = m_cr(data)  # 0.5
print(res)

# 返回基于LLM计算的召回率，LLM基于answer和context_retrieved判断是否召回了所有相关文档
# 适用于没有标注的情况下使用，比较耗费 token ，请根据需求谨慎使用
m_lcr = LLMContextRecall(
    lazyllm.OnlineChatModule(
        source="qwen",
        model="qwen-plus-latest",
        api_key=os.getenv("QWEN_API_KEY"),
    )
)
res = m_lcr(data)  # 1.0
print(res)

我们自己定义了一个非常简单的数据集。`context_retrieved`表示**实际被召回**的段落，`context_reference` 表示标注的**应当被召回**的段落。

LazyLLM 在评估召回率上有两种方式，一种是`NonLLMContextRecall`方法，这种情况不调用大语言模型计算，由于`context_retrieved` 包含了 `context_reference` 中的所有内容，因此所有相关的上下文都被成功召回了，召回率为 1。

另一种方式是`LLMContextRecall`方法，这种情况会调用大语言模型，基于`answer`和`context_retrieved`判断是否召回了所有相关文档。大致流程是这样的：

1. 输入数据 ：需要包含 `question` （问题）、 `answer` （标准答案）和 `context_retrieved` （检索器召回的文档）字段。

2. 处理过程 ：

   1. 将所有召回的文档内容连接成一个字符串作为上下文。
   2. 构造一个查询字符串，包含问题、上下文和标准答案。
   3. 使用大语言模型（LLM）对这个查询进行评估，判断答案中的每个句子是否可以归因于给定的上下文。

3. 评估逻辑 ：

   1. LLM 会分析答案中的每个句子，并给出二元评分（1 表示完全支持，0 表示不支持/矛盾）。评估结果是一个包含每个句子评分的列表。

4. 计算召回率 ：计算所有句子评分的平均值，即为最终的召回率。

同样由于 `context_retrieved` 包含了 `context_reference` 的所有内容，因此 LLM 判断所有句子都得到了支持，召回率为 1.0。

计算召回率 ：

上下文相关性分数`ContextRelevance`就更好理解了，`context_retrieved`里面与问答相关的只有一句话，所以分数为 0.5。

当然了，这个数据集是我们自定义的。如果使用真实的数据集呢？我们用第二节实现的基于 [cmrc2018](https://huggingface.co/datasets/LazyAGI/CMRC2018_Knowledge_Base/tree/main) 数据集做的 RAG 系统进行评测，以下是评测代码：

In [ ]:
# 导入所需的库
import lazyllm
import os
# 导入评估工具：LLMContextRecall（基于大语言模型的上下文召回率）、NonLLMContextRecall（非大语言模型的上下文召回率）、ContextRelevance（上下文相关性）
from lazyllm.tools.eval import LLMContextRecall, NonLLMContextRecall, ContextRelevance
# 导入Hugging Face datasets库，用于加载数据集
from datasets import load_dataset

# 加载cmrc2018数据集，指定缓存目录为当前目录下的datasets文件夹
dataset = load_dataset("cmrc2018", cache_dir="./datasets")


def create_evaluation_data(test_dataset, topk_values=[1, 3, 5]):
    """创建用于评估的数据结构

    Args:
        test_dataset: 测试数据集
        topk_values: 检索器返回的topk值（召回的文档数）列表，默认为[1, 3, 5]

    Returns:
        dict: 包含不同topk值评估数据的字典
    """
    # 文档加载，将data_kb目录作为知识库
    documents = lazyllm.Document(dataset_path="./data_kb")

    # 初始化评估数据字典
    evaluation_data = {}

    # 为每个topk值创建评估数据
    for topk in topk_values:
        # 检索组件定义
        # doc: 指定文档对象
        # group_name: 指定文档分组方式为"CoarseChunk"(粗粒度分块)
        # similarity: 使用"bm25_chinese"算法计算相似度，适合中文检索
        # topk: 返回最相关的topk个结果
        retriever = lazyllm.Retriever(
            doc=documents, group_name="CoarseChunk", similarity="bm25_chinese", topk=topk
        )

        # 初始化数据列表
        data = []
        # 使用测试集的前10个样本进行评估
        for i in range(min(10, len(test_dataset))):
            # 获取当前样本
            sample = test_dataset[i]
            # 提取问题
            question = sample["question"]
            # 提取答案，取第一个答案作为标准答案
            answer = sample["answers"]["text"][0]
            # 提取标注的应当被召回的段落
            context_reference = [sample["context"]]

            # 获取检索器召回的文档
            doc_node_list = retriever(query=question)
            # 提取召回文档的内容
            context_retrieved = [node.get_content() for node in doc_node_list]

            # 将数据添加到列表中
            data.append({
                "question": question,
                "answer": answer,
                "context_retrieved": context_retrieved,
                "context_reference": context_reference,
            })

        # 将当前topk值的数据添加到评估数据字典中
        evaluation_data[topk] = data

    return evaluation_data


def evaluate_rag_performance(evaluation_data):
    """评估RAG性能

    Args:
        evaluation_data: 包含不同topk值评估数据的字典

    Returns:
        dict: 包含不同topk值评估结果的字典
    """
    # 初始化评估工具
    # NonLLMContextRecall: 计算召回文档的命中率
    m_recall = NonLLMContextRecall()
    # ContextRelevance: 计算召回文档中的上下文相关性分数
    m_cr = ContextRelevance()
    # LLMContextRecall: 计算基于LLM的召回率
    m_lcr = LLMContextRecall(
        lazyllm.OnlineChatModule(
            source="qwen",
            model="qwen-plus-latest",
            api_key=os.getenv("QWEN_API_KEY"),
        )
    )

    # 初始化结果字典
    results = {}

    # 对每个topk值进行评估
    for topk, data in evaluation_data.items():
        print(f"\n=== TopK = {topk} 评估结果 ===")

        # 计算召回文档的命中率
        recall_score = m_recall(data)
        print(f"召回率 (Recall): {recall_score}")

        # 计算召回文档中的上下文相关性分数
        relevance_score = m_cr(data)
        print(f"上下文相关性 (Context Relevance): {relevance_score}")

        # 计算基于LLM的召回率
        try:
            llm_recall_score = m_lcr(data)
            print(f"基于LLM的召回率 (LLM Context Recall): {llm_recall_score}")
        except Exception as e:
            print(f"计算基于LLM的召回率时出错: {e}")
            llm_recall_score = 0.0

        # 将结果添加到结果字典中
        results[topk] = {
            "recall": recall_score,
            "relevance": relevance_score,
            "llm_recall": llm_recall_score
        }

    return results


def main():
    # 创建评估数据
    print("正在创建评估数据...")
    evaluation_data = create_evaluation_data(dataset["test"], topk_values=[1, 3, 5])

    # 评估RAG性能
    print("开始评估RAG性能...")
    results = evaluate_rag_performance(evaluation_data)

    # 打印最终结果
    print("\n=== 最终评估结果 ===")
    for topk, scores in results.items():
        print(f"\nTopK = {topk}:")
        print(f"  召回率: {scores['recall']:.4f}")
        print(f"  上下文相关性: {scores['relevance']:.4f}")
        print(f"  基于LLM的召回率: {scores['llm_recall']:.4f}")


if __name__ == "__main__":
    main()

我看了下 [cmrc2018](https://huggingface.co/datasets/LazyAGI/CMRC2018_Knowledge_Base/tree/main) 数据集的测试结果，还是很满意的。

| TopK 值 | 召回率 | 上下文相关性 | 基于 LLM 的召回率 |
| ------- | ------ | ------------ | ----------------- |
| 1       | 0.9000 | 1.0000       | 1.0000            |
| 3       | 1.0000 | 0.3567       | 0.9000            |
| 5       | 1.0000 | 0.2161       | 0.9000            |

不知道有没有发现这样一个现象，随着召回文档数量增加，相关性反而从 100%掉到 21.6%了。这就像是**为了找一个答案，把整个图书馆的书都搬过来，结果真正有用的反而被淹没了**。

这种情况我太熟悉了。刚开始做 RAG 的时候，总想着多召回一些文档总没错吧？结果发现不是这么回事儿。**垃圾信息太多，大模型也会被带偏**。有一次我们的系统，用户问"如何做西红柿炒蛋"，结果因为召回了太多无关内容，最后回答里居然出现了"西红柿的种植技术"...

所以现在做 RAG，不能光追求召回率，相关性同样重要。这就像做菜，不是食材越多越好，而是要精准。宁可少而精，也不要多而杂。当然，怎么在两者之间找平衡，这又是另一个让人头疼的问题了。